In [1]:
import numpy as np
np.set_printoptions(precision=2, floatmode="fixed")

In [2]:
def row_swap(mat, permutation):
    first_col = mat[:, 0]
    nonzero_elems = np.nonzero(first_col)
    # if there are no nonzero rows, end
    if nonzero_elems[0].size == 0:
        return False
    # if the first row is not nonzero, swap
    if nonzero_elems[0][0] != 0:
        mat[[0, nonzero_elems[0][0]], :] = mat[[nonzero_elems[0][0], 0], :]
        permutation[[0, nonzero_elems[0][0]], :] = permutation[[nonzero_elems[0][0], 0], :]
    # else the matrix is good, all non zero
    return True

In [3]:
def recursive(mat, orig_shape, permutation):
    if (mat.shape == (1, 1)):
        l = np.zeros(orig_shape[0]) # n-vector
        l[-1] = 1
        l = np.transpose([l])
        u = np.zeros(orig_shape[1]) # m-vector
        u[-1] = mat[0, 0]
        return l, u
    
    row_vector = mat[0, :]
    column_vector = None
    
    # If we couldn't row swap, it's the final recursive step
    if not row_swap(mat, permutation):
        arr = np.zeros(mat.shape[0])
        arr[0] = 1
        column_vector = np.transpose([arr])
    else:
        column_vector = mat[:, 0] / row_vector[0, 0]
    
    remainder_matrix = mat - column_vector @ row_vector
    
    shape = column_vector.shape
    padding_dim = orig_shape[0] - shape[0]
    
    padded_column = np.pad(column_vector, [ (padding_dim, 0), (0, 0) ], mode='constant')
    padded_row    = np.pad(row_vector,    [ (0, 0), (padding_dim, 0) ], mode='constant')
    
    lower, upper = padded_column, padded_row
    
    # if the remainder matrix is not all zeros
    if (np.any(remainder_matrix)):
        truncated_remainder = remainder_matrix[1:,1:]
        l, u = recursive(truncated_remainder, orig_shape, permutation)
        lower = np.hstack((lower, l))
        upper = np.vstack((upper, u))
        
    
    return lower, upper

In [4]:
def decompose(matrix):
    mat = matrix.copy()
    permutation = np.identity(mat.shape[0])
    lower, upper = recursive(mat, mat.shape, permutation)
    return permutation, lower, upper

In [11]:
# matrix = np.matrix('0 -2 -3; 4 -3 -4; -6 7 14')
# print("original")
# print(matrix)
# perm, lower, upper = decompose(matrix)
# print("lower")
# print(lower)
# print("upper")
# print(upper)
# print("perm")
# print(perm)
# print("check")
# mult = perm @ lower @ upper
# print(mult)
# print(np.array_equal(mult, matrix))

In [6]:
def test(matrix):
    print("original")
    print(matrix)
    perm, lower, upper = decompose(matrix)
    print("lower")
    print(lower)
    print("upper")
    print(upper)
    print("perm")
    print(perm.astype(int))
    mult = perm @ lower @ upper
    print("check")
    print(mult.astype(int))
    print(">>>>", np.array_equal(mult, matrix))
    return "-"*20

In [10]:
# print(test(np.matrix('0 -2 -3; 4 -3 -4; -6 7 14')))
# print(test(np.matrix('-2 1 3; -4 4 1')))
# print(test(np.matrix('-2 1; 4 3; 8 6')))
# print(test(np.matrix('-2 2 3; -4 4 1')))
# print(test(np.matrix('4 2 3; -4 4 1')))
# print(test(np.matrix('0 43 45 514; 0 94 5 5; 2 5 5 3; 3 4 5 3')))

In [8]:
print(test(np.matrix('2 4 2 0; 1 1 -1 -2; -2 -2 3 4; 3 7 5 2')))

original
[[ 2  4  2  0]
 [ 1  1 -1 -2]
 [-2 -2  3  4]
 [ 3  7  5  2]]
lower
[[ 1.00  0.00  0.00]
 [ 0.50  1.00  0.00]
 [-1.00 -2.00  1.00]
 [ 1.50 -1.00  0.00]]
upper
[[ 2.00  4.00  2.00  0.00]
 [ 0.00 -1.00 -2.00 -2.00]
 [ 0.00  0.00  1.00  0.00]]
perm
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
check
[[ 2  4  2  0]
 [ 1  1 -1 -2]
 [-2 -2  3  4]
 [ 3  7  5  2]]
>>>> True
--------------------
